### Setup

In [1]:
import sklearn as sk
import numpy as np
import pandas as pd

#this is the comment
#this is another comment
#another comment
#addin another comment

## Import Data

In [2]:
df = pd.read_csv('/mnt/data/prepared.csv', header=0, index_col=0)
print(len(df), " rows")
print(len(df.columns), " cols")
df.head()

7939  rows
17  cols


,dropperc,mins,KPI_DeviceSat,consecmonths,long,income,age,KPI_NetworkQual,dataperc,datause,bill,calls,education,servcalls,dataplan,kids,churn_Y
custid,,,,,,,,,,,,,,,,,
844336,0.016364,550,0.016035,28,35,89.2,45,0.760355,0.158936,651,71.44,55,11,0,4096,2,0
146041,0.018349,545,0.017055,33,35,54.2,43,0.526147,0.377930,774,124.23,66,9,0,2048,1,0
847745,0.018519,378,0.008896,41,19,55.3,41,0.798811,0.282227,578,112.42,91,12,0,2048,1,0
285565,0.014493,552,0.016460,32,38,66.8,31,0.631075,0.271240,1111,105.12,71,10,0,4096,0,0
754611,0.012132,577,0.020711,4,31,87.2,43,0.726428,0.198730,407,109.48,56,8,0,2048,0,0


### Begin Model Building

In [3]:
columns = list(df.columns)
columns.remove('churn_Y')
y = df["churn_Y"].values
X = df[columns].values

In [4]:
from sklearn.ensemble import AdaBoostClassifier
ad1 = AdaBoostClassifier(learning_rate=1)
ad1 = ad1.fit(X, y)
ad1prb = ad1.predict_proba(X)

In [5]:
from sklearn.ensemble import AdaBoostClassifier
ad2 = AdaBoostClassifier(learning_rate=0.5)
ad2 = ad2.fit(X, y)
ad2prb = ad2.predict_proba(X)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
gb1 = GradientBoostingClassifier(loss = 'exponential', max_depth=3)
gb1 = gb1.fit(X, y)
gb1prb = gb1.predict_proba(X)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb2 = GradientBoostingClassifier(loss = 'exponential', max_depth=10)
gb2 = gb2.fit(X, y)
gb2prb = gb2.predict_proba(X)

In [8]:
from sklearn.ensemble import RandomForestClassifier 
rf1 = RandomForestClassifier(max_depth=None)
rf1 = rf1.fit(X, y)
rf1prb = rf1.predict_proba(X)
rf1pclass = rf1.predict(X)

In [9]:
from sklearn.ensemble import RandomForestClassifier 
rf2 = RandomForestClassifier(max_depth = 5)
rf2 = rf2.fit(X, y)
rf2prb = rf2.predict_proba(X)
rf2pclass = rf2.predict(X)

In [10]:
from sklearn.ensemble import RandomForestClassifier 
rf3 = RandomForestClassifier(max_depth = 30)
rf3 = rf3.fit(X, y)
rf3prb = rf3.predict_proba(X)
rf3pclass = rf3.predict(X)

### Build a Cross Validation Function

In [11]:
#from sklearn import cross_validation
#cross_validation module depricated in 0.18, possibly will be removed in 0.20
#same function now available in model_selection module
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

# takes a list of models, the input np.array, the target np.array, the type of score to be used with cv, and k
# each element in the list of models should have two items: the model object and the name you want to use for that 
# model object
# returns a dataframe with the names you entered and the mean of the cv scores across all k folds

def cv_fun(models, inputs, target, score, k):
    i = 0
    for m in models:
        #scores = cross_validation.cross_val_score(models[i][0], inputs, target, scoring=score, cv=k)
        scores = cross_val_score(models[i][0], inputs, target, scoring=score, cv=k)
        if i==0:
            list1 = list()
            list2 = list()
            
        list1.append(round(scores.mean(),3))
        list2.append(models[i][1])

        i=i+1

    return pd.DataFrame(list1, index=list2, columns=[score])

### Cross Validation on the Training Data

In [12]:
# use the cv function found up under the Setup section
# enter a list with each entry holding the model object followed by a text name you want to give the model

input_models = [[ad1, 'ad1']]
input_models.append([ad2, 'ad2'])
input_models.append([gb1, 'gb1'])
input_models.append([gb2, 'gb2'])
input_models.append([rf1, 'rf1'])
input_models.append([rf2, 'rf2'])
input_models.append([rf3, 'rf3'])

cv_auc = cv_fun(input_models, X, y, 'roc_auc', 5)
cv_acc = cv_fun(input_models, X, y, 'accuracy', 5)
cv_results = cv_auc.join(cv_acc)
cv_results.sort_values(by='roc_auc', ascending=False, inplace=True)

In [13]:
#manually pick best, would be nice to automate this 
cv_results

,roc_auc,accuracy
ad1,0.985,0.976
ad2,0.984,0.970
gb1,0.983,0.971
gb2,0.972,0.970
rf2,0.946,0.954
rf3,0.940,0.964
rf1,0.939,0.963


In [14]:
#automate model selection.. probably not the most efficient way to do this

for i,_ in enumerate(input_models):
    if input_models[i][1] == cv_results.index.values[0]:
        #print(input_models[i][1])
        #print(_[0])
        best_model, model_name = _
    else:
        continue
    
#selected[0].predict(X)


In [15]:
print(best_model, model_name)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=50, random_state=None) ad1


In [16]:
#cv_results.index.values[0]

### Save Best Model to File

In [17]:
import pickle 

# save best model
file = '/mnt/output/selected.pkl'
pickle.dump(best_model, open(file, 'wb')) # w = open for writing, r = open for reading, b = binary

In [18]:

#make confusion matrix plot
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot
 
    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
 
    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']
 
    title:        the text to display at the top of the matrix
 
    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues
 
    normalize:    If False, plot the raw numbers
                  If True, plot the proportions
 
    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph
 
    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
 
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
 
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
 
    if cmap is None:
        cmap = plt.get_cmap('Blues')
 
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
 
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
 
 
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
 
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.gcf().subplots_adjust(bottom=0.25)
    plt.savefig('/mnt/output/ConfMatx_Best.png', format='png')
    plt.show()
    #plt.gcf().clear()
    


In [19]:
from sklearn import metrics

plot_confusion_matrix(cm           = metrics.confusion_matrix(y, best_model.predict(X)), 
                      normalize    = False,
                      target_names = ['no churn', 'churn'],
                      title        = "Confusion Matrix for Best Model")
                      

<Figure size 800x600 with 2 Axes>

In [20]:
df2 = df
#note slight mod to line below to pull model object from 'selected' variable. 2nd item in list contains name. 

df2["prob"] = best_model.predict_proba(X)[:,1]
df2.to_csv('/mnt/data/modelOut.csv')

In [21]:
df2.head()

,dropperc,mins,KPI_DeviceSat,consecmonths,long,income,age,KPI_NetworkQual,dataperc,datause,bill,calls,education,servcalls,dataplan,kids,churn_Y,prob
custid,,,,,,,,,,,,,,,,,,
844336,0.016364,550,0.016035,28,35,89.2,45,0.760355,0.158936,651,71.44,55,11,0,4096,2,0,0.451429
146041,0.018349,545,0.017055,33,35,54.2,43,0.526147,0.377930,774,124.23,66,9,0,2048,1,0,0.426584
847745,0.018519,378,0.008896,41,19,55.3,41,0.798811,0.282227,578,112.42,91,12,0,2048,1,0,0.417519
285565,0.014493,552,0.016460,32,38,66.8,31,0.631075,0.271240,1111,105.12,71,10,0,4096,0,0,0.442955
754611,0.012132,577,0.020711,4,31,87.2,43,0.726428,0.198730,407,109.48,56,8,0,2048,0,0,0.252558


In [22]:
df2.shape

(7939, 18)